In [2]:
import openai 
import pandas as pd
import re
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
import os
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import pandas as pd
import logging
from typing import Optional
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=openai_api_key)

# Información relacionada a los videos 

## Hashtags ideales

In [3]:
df=pd.read_csv("df_processed.csv")

def hashtags_ideales(
    df: pd.DataFrame,
    api_key: str,
    cantidad= int,
    modelo: str = "gpt-4o-mini",
    filtro = int,  
    temperatura: float = 0.7  # Con este si funciona, no entiendo pq es medio rarito
) -> Optional[str]:
    try:
        # Verificar si existe la columna 'playCount'
        if 'playCount' not in df.columns:
            raise ValueError("El DataFrame debe contener una columna 'playCount'")
        
        if 'hashtags' not in df.columns:
            raise ValueError("El DataFrame debe contener una columna 'hashtags'")
        
        # esto porque no permite mucha carga 
        df_muestra = df[df["playCount"] > filtro]        
        # df_muestra['contenido_limpio'] = df_muestra['contenido_limpio']
            
        agent = create_pandas_dataframe_agent(
            ChatOpenAI(
                temperature=temperatura,
                model=modelo,
                openai_api_key=api_key,
                max_tokens=2000  
            ),
            df_muestra,
            verbose=True,
            agent_type=AgentType.OPENAI_FUNCTIONS,
            allow_dangerous_code=True
        )
        
        prompt = f"""
        Necesito que analises las siguientes columnas de mi dataframe: 'hashtags' y 'playCount'
        Con este analisis dame la cantidad {cantidad} de hashtags ideales para mis videos basandote en tu investigación

        
        """
        
        resultado = agent.invoke({
            "input": prompt
        })
        
        return resultado.get('output', "No se pudo generar una respuesta")
        
    except Exception as e:
        logging.error(f"Error en crear_idea_guion: {str(e)}")
        return f"Error: {str(e)}"
    
try:
        print("\n--- Investigando Hashtags ideales ---\n")
        
        hash = hashtags_ideales(
            df=df,
            cantidad=4,
            api_key=openai_api_key,
            filtro=10000
        )
        print("\nHashtags ideales:")
        print(hash)
        
except Exception as e:
        
        print(f"Error al ejecutar el script: {str(e)}")


--- Investigando Hashtags ideales ---



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nfrom collections import Counter\n\n# Supongamos que df es tu dataframe\n\n# Contar los hashtags\nhashtag_list = df['hashtags'].str.cat(sep=' ').split()\n# Contar la frecuencia de cada hashtag\nhashtag_counts = Counter(hashtag_list)\n\n# Crear un DataFrame de los hashtags y sus conteos\nhashtag_counts_df = pd.DataFrame(hashtag_counts.items(), columns=['hashtag', 'count'])\n\n# Unir los hashtags con los playCounts correspondientes\nhashtag_playcount = df.explode('hashtags')\n\n# Agrupar por hashtag y sumar los playCounts\nhashtag_playcount_sum = hashtag_playcount.groupby('hashtags')['playCount'].sum().reset_index()\n\n# Unir el conteo de hashtags y el playCount\nresult = pd.merge(hashtag_counts_df, hashtag_playcount_sum, left_on='hashtag', right_on='hashtags', how='left')\n\n# Ordenar por playCount y count para encontrar los hashtags ideales\n

## Ideas Principales

In [4]:
df=pd.read_csv("df_processed.csv")

def ideales_principales(
    df: pd.DataFrame,
    api_key: str,
    cantidad= int,
    modelo: str = "gpt-4o-mini",
    filtro = int,  
    temperatura: float = 0.7  # Con este si funciona, no entiendo pq es medio rarito
) -> Optional[str]:
    try:
        # Verificar si existe la columna 'playCount'
        if 'playCount' not in df.columns:
            raise ValueError("El DataFrame debe contener una columna 'playCount'")
        
        
        # esto porque no permite mucha carga 
        df_muestra = df[df["playCount"] > filtro]        
        # df_muestra['contenido_limpio'] = df_muestra['contenido_limpio']
            
        agent = create_pandas_dataframe_agent(
            ChatOpenAI(
                temperature=temperatura,
                model=modelo,
                openai_api_key=api_key,
                max_tokens=2000  
            ),
            df_muestra,
            verbose=True,
            agent_type=AgentType.OPENAI_FUNCTIONS,
            allow_dangerous_code=True
        )
        
        prompt = f"""
        Necesito que analises las siguientes columnas de mi dataframe: 'text'
        quiero que analisis esta columna de mi df y con eso por favor dame este nuemro de ideas principales {cantidad}

        
        """
        
        resultado = agent.invoke({
            "input": prompt
        })
        
        return resultado.get('output', "No se pudo generar una respuesta")
        
    except Exception as e:
        logging.error(f"Error en crear_idea_guion: {str(e)}")
        return f"Error: {str(e)}"
    
try:
        print("\n--- Investigando Ideas principales ---\n")
        
        idea = ideales_principales(
            df=df,
            cantidad=4,
            api_key=openai_api_key,
            filtro=10000
        )
        print("\nIdeas principales:")
        print(idea)
        
except Exception as e:
        print(f"Error al ejecutar el script: {str(e)}")


--- Investigando Ideas principales ---



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\n# Suponiendo que df es el dataframe que mencionaste\ntext_data = df['text']\n\n# Análisis simple para extraer ideas principales\nideas_principales = text_data.str.extractall(r'([A-Za-z0-9\\s]+)')\nideas_principales = ideas_principales[0].value_counts().head(4)\n\nideas_principales"}`


agenda asesoria gratuita prepitcommx                                                                                                                                                                                                                     3
n                                                                                                                                                                                                                                                        2
si hijo apasiona tecnologia inscribelo curso verano corre lin

## Descripcion ideal

In [5]:
df=pd.read_csv("df_processed.csv")

def descripcion(
    df: pd.DataFrame,
    api_key: str,
    tema:str,
    modelo: str = "gpt-4o-mini",
    filtro = int,  
    temperatura: float = 0.7  # Con este si funciona, no entiendo pq es medio rarito
) -> Optional[str]:
    try:
        # Verificar si existe la columna 'playCount'
        if 'playCount' not in df.columns:
            raise ValueError("El DataFrame debe contener una columna 'playCount'")
        
        
        # esto porque no permite mucha carga 
        df_muestra = df[df["playCount"] > filtro]        
        # df_muestra['contenido_limpio'] = df_muestra['contenido_limpio']
            
        agent = create_pandas_dataframe_agent(
            ChatOpenAI(
                temperature=temperatura,
                model=modelo,
                openai_api_key=api_key,
                max_tokens=2000  
            ),
            df_muestra,
            verbose=True,
            agent_type=AgentType.OPENAI_FUNCTIONS,
            allow_dangerous_code=True
        )
        
        prompt = f"""
        Necesito que analises las siguientes columnas de mi dataframe: 'text'
        Y me des la descripcion ideal para un video de tiktok sobre este tema {tema}, acuerdate que una descripcion es breve
        """
        
        resultado = agent.invoke({
            "input": prompt
        })
        
        return resultado.get('output', "No se pudo generar una respuesta")
        
    except Exception as e:
        logging.error(f"Error en crear_idea_guion: {str(e)}")
        return f"Error: {str(e)}"
    
try:
        print("\n--- Investigando Descripción ideal ---\n")
        
        descripcion = descripcion(
            df=df,
            tema="educacion universitaria",
            api_key=openai_api_key,
            filtro=1000
        )
        print("\nIDescripcion ideal:")
        print(descripcion)
        
except Exception as e:
        print(f"Error al ejecutar el script: {str(e)}")


--- Investigando Descripción ideal ---



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Extraer la columna 'text' del dataframe y analizar su contenido\ntext_data = df['text']\n\n# Mostrar una muestra de los textos para análisis\ntext_data_sample = text_data.sample(5)\ntext_data_sample"}`


581    cuales profesiones ma mal pagas colombia aqui ...
149    cosas deberia saber si gustaria estudiar zoote...
433    gustaria estudiar emprendimiento corea dia hoy...
460    cuatro terminos derecho administrativo explica...
181                     quieres paguen estudiar posgrado
Name: text, dtype: objectHe analizado algunos textos del dataframe relacionados con la educación universitaria. A continuación, te propongo una descripción breve para un video de TikTok sobre este tema:

---

🎓 ¿Estás listo para dar el siguiente paso en tu educación? Descubre las oportunidades que te ofrecen los programas de posgrado y cómo pueden transformar tu carrera. Desde es

# Documentación
Este cuaderno (archivo .ipynb) está diseñado para procesar datos relacionados con contenido de videos, utilizando herramientas como OpenAI y pandas para análisis y generación de texto. A continuación, se describe su estructura, funciones principales y propósitos.

# 1. Bibliotecas utilizadas

El archivo importa las siguientes bibliotecas:
- openai: Interacción con la API de OpenAI.
- pandas: Manipulación y análisis de datos.
- re: Expresiones regulares para manejo de texto.
- typing: Uso de tipos opcionales para las funciones.

# Funciones usadas
## Función: Hashtags ideales
### Propósito
Esta función analiza un conjunto de datos para generar hashtags ideales para los videos, utilizando la API de OpenAI.

### Entrada
- df: Un DataFrame de pandas que contiene información sobre los videos.
- api_key: Clave de la API de OpenAI.
- Cantidad: número de hashtags que se devuelven.
- Filtro: número de vistas que funciona como filtro para que se escoga solo los videos que supera ese numero de vistas.

### Salida 
Retorna un conjunto de hashtags ideales basados en el contenido del DataFrame.

## Función:Ideas Principales
### Propósito
Extrae ideas relevantes o principales del contenido de los videos.

### Entrada
- df: DataFrame con los datos procesados.
- api_key: Clave para la API de OpenAI.
- Cantidad: número de ideas que se retornaran
- Filtro: número de vistas que funciona como filtro para que se escoga solo los videos que supera ese numero de vistas.

## Función: Descripción ideal
### Propósito
Crea una descripción óptima para los videos, posiblemente basada en datos como palabras clave, títulos o contenido relacionado.

### Entrada
- df: DataFrame con datos relacionados al video.
- api_key: Clave API de OpenAI.
- Filtro: número de vistas que funciona como filtro para que se escoga solo los videos que supera ese numero de vistas.
- Tema: a que va aorientado tu video para que ayude en la creación de la descripción.